In [2]:
import csv
import random

def is_winner(board, player):
    # Check rows, columns, and diagonals
    for i in range(3):
        if all(board[i][j] == player for j in range(3)) or \
           all(board[j][i] == player for j in range(3)):
            return True
    if all(board[i][i] == player for i in range(3)) or \
       all(board[i][2-i] == player for i in range(3)):
        return True
    return False

def get_empty_cells(board):
    return [(i, j) for i in range(3) for j in range(3) if board[i][j] == ' ']

def generate_tictactoe_logs(num_games, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['event_type', 'agent_id', 'context']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for _ in range(num_games):
            board = [[' ' for _ in range(3)] for _ in range(3)]
            
            # Game Start
            writer.writerow({
                'event_type': 'GAME_START',
                'agent_id': 'system',
                'context': 'New Game'
            })

            current_player = 'X'
            game_result = None

            while True:
                empty_cells = get_empty_cells(board)
                if not empty_cells:
                    game_result = 'draw'
                    break

                move = random.choice(empty_cells)
                board[move[0]][move[1]] = current_player

                writer.writerow({
                    'event_type': 'MOVE',
                    'agent_id': current_player,
                    'context': f"{move[0]},{move[1]}"
                })

                if is_winner(board, current_player):
                    game_result = current_player
                    break

                current_player = 'O' if current_player == 'X' else 'X'

            # Game End
            writer.writerow({
                'event_type': 'GAME_END',
                'agent_id': 'system',
                'context': game_result
            })

    print(f"Generated logs for {num_games} Tic Tac Toe games. Saved to {output_file}")

In [3]:
generate_tictactoe_logs(num_games=1000, output_file='../../../processed/games/tic-tac-toe/1k_single_agent.csv')

Generated logs for 1000 Tic Tac Toe games. Saved to ../../../processed/games/tic-tac-toe/1k_single_agent.csv


In [4]:
generate_tictactoe_logs(num_games=1000 * 10, output_file='../../../processed/games/tic-tac-toe/10k_single_agent.csv')

Generated logs for 10000 Tic Tac Toe games. Saved to ../../../processed/games/tic-tac-toe/10k_single_agent.csv


In [5]:
generate_tictactoe_logs(num_games=1000 * 50, output_file='../../../processed/games/tic-tac-toe/50k_single_agent.csv')

Generated logs for 50000 Tic Tac Toe games. Saved to ../../../processed/games/tic-tac-toe/50k_single_agent.csv


In [9]:
import csv
import random

def is_winner(board, player):
    for i in range(3):
        if all(board[i][j] == player for j in range(3)) or \
           all(board[j][i] == player for j in range(3)):
            return True
    if all(board[i][i] == player for i in range(3)) or \
       all(board[i][2-i] == player for i in range(3)):
        return True
    return False

def get_empty_cells(board):
    return [(i, j) for i in range(3) for j in range(3) if board[i][j] == ' ']

def minimax(board, depth, is_maximizing, player, opponent):
    if is_winner(board, player):
        return 10 - depth
    if is_winner(board, opponent):
        return depth - 10
    if not get_empty_cells(board):
        return 0

    if is_maximizing:
        best_score = float('-inf')
        for i, j in get_empty_cells(board):
            board[i][j] = player
            score = minimax(board, depth + 1, False, player, opponent)
            board[i][j] = ' '
            best_score = max(score, best_score)
        return best_score
    else:
        best_score = float('inf')
        for i, j in get_empty_cells(board):
            board[i][j] = opponent
            score = minimax(board, depth + 1, True, player, opponent)
            board[i][j] = ' '
            best_score = min(score, best_score)
        return best_score

def get_best_move(board, player):
    opponent = 'O' if player == 'X' else 'X'
    best_score = float('-inf')
    best_moves = []
    for i, j in get_empty_cells(board):
        board[i][j] = player
        score = minimax(board, 0, False, player, opponent)
        board[i][j] = ' '
        if score > best_score:
            best_score = score
            best_moves = [(i, j)]
        elif score == best_score:
            best_moves.append((i, j))
    return random.choice(best_moves)

def play_game(writer):
    board = [[' ' for _ in range(3)] for _ in range(3)]
    writer.writerow({
        'event_type': 'GAME_START',
        'agent_id': 'system',
        'context': 'New Game'
    })
    current_player = 'X'
    moves_made = 0

    while True:
        empty_cells = get_empty_cells(board)
        if not empty_cells:
            return 'draw'

        if moves_made < 3:
            move = random.choice(empty_cells)
        else:
            if random.random() < 0.9:  # 90% chance of making the best move
                move = get_best_move(board, current_player)
            else:
                move = random.choice(empty_cells)

        board[move[0]][move[1]] = current_player
        moves_made += 1

        writer.writerow({
            'event_type': 'MOVE',
            'agent_id': current_player,
            'context': f"{move[0]},{move[1]}"
        })

        if is_winner(board, current_player):
            return current_player

        current_player = 'O' if current_player == 'X' else 'X'

def generate_tictactoe_logs(num_games, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['event_type', 'agent_id', 'context']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        outcomes = {'X': 0, 'O': 0, 'draw': 0}
        target_outcomes = num_games // 3

        while outcomes['X'] < target_outcomes or outcomes['O'] < target_outcomes or outcomes['draw'] < target_outcomes:
            result = play_game(writer)
            if outcomes[result] < target_outcomes:
                outcomes[result] += 1
                writer.writerow({
                    'event_type': 'GAME_END',
                    'agent_id': 'system',
                    'context': result
                })

    print(f"Generated logs for {num_games} Tic Tac Toe games. Saved to {output_file}")
    print(f"Outcomes: X wins: {outcomes['X']}, O wins: {outcomes['O']}, Draws: {outcomes['draw']}")



In [10]:
generate_tictactoe_logs(num_games=1000, output_file='../../../../1k_single_agent_minmax.csv')

Generated logs for 1000 Tic Tac Toe games. Saved to ../../../../1k_single_agent_minmax.csv
Outcomes: X wins: 235, O wins: 210, Draws: 555
